In [12]:
from prompt_templates import TASK_PROMPT
import pandas as pd


df = pd.read_csv("../data/eval/dependencies_without_rules.csv", sep=";")

for x in df.to_dict("records"):
    print(TASK_PROMPT.format(
        nameA=x["option_name"],
        typeA=x["option_type"],
        valueA=x["option_value"],
        fileA=x["option_file"],
        technologyA=x["option_technology"],
        nameB=x["dependent_option_name"],
        typeB=x["dependent_option_type"],
        valueB=x["dependent_option_value"],
        fileB=x["dependent_option_file"],
        technologyB=x["dependent_option_technology"],
    ))


Carefully evaluate whether configuration option spring.datasource.druid.stat-view-servlet.login-password of type ConfigType.PASSWORD with value druid in mall-admin/src/main/resources/application-dev.yml of technology spring     depends on configuration option spring.datasource.druid.stat-view-servlet.login-password of type ConfigType.PASSWORD with value druid in mall-search/src/main/resources/application-prod.yml of technology spring or vice versa.
Carefully evaluate whether configuration option project.groupId of type ConfigType.NAME with value com.macro.mall in mall-mbg/pom.xml of technology maven     depends on configuration option project.parent_mall.groupId of type ConfigType.NAME with value com.macro.mall in mall-mbg/pom.xml of technology maven or vice versa.
Carefully evaluate whether configuration option spring.h2.console.enabled of type ConfigType.BOOLEAN with value true in apollo-biz/src/test/resources/application.properties of technology spring     depends on configuration o

In [3]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env")

pinecone_client = Pinecone(api_key=os.getenv(key="PINECONE_API_KEY"))

#index = pinecone_client.Index("github")

#if index:
#    print(index.describe_index_stats())
#    print(index.describe_index_stats()["total_vector_count"])

In [4]:
from ingestion import IngestionEngine

engine = IngestionEngine(
    pinecone_client=pinecone_client,
    splitting="sentence",
    extractors=[]
)

docs = engine.docs_from_github(project_name="apollo")

print(len(docs))

[06/25/24 08:43:42] INFO     Ingestion engine initialized.                                          ]8;id=195289;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=940522;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     Start scraping the repository of apollo.                              ]8;id=745198;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=891293;file:///home/simisimon/GitHub/cval/src/ingestion.py#180\180]8;;\

apolloconfig
master


                    INFO     HTTP Request: GET                                                      ]8;id=27092;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=525141;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/branches/master                      
                             "HTTP/1.1 200 OK"                                                                     

[06/25/24 08:43:43] INFO     HTTP Request: GET                                                      ]8;id=58061;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=677507;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/git/trees/26829dd57f3                
                             ae275d4eb7627971decfd7ee90699 "HTTP/1.1 200 OK"                                       

...

                    INFO     HTTP Request: GET                                                      ]8;id=821647;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=665934;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/git/blobs/26e36ba5a91                
                             11ac72cb49d8d2ed5deb6dc2adb36 "HTTP/1.1 200 OK"                                       

                    INFO     HTTP Request: GET                                                      ]8;id=665947;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=150910;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/git/blobs/1f79ed9c976                
                             5e793e572e644f6df3714f638b96d "HTTP/1.1 200 OK"                                       

                    INFO     HTTP Request: GET                                                      ]8;id=611305;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=771533;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/git/blobs/28ec5cc0fbc                
                             b6b3b5f8a7c24e8de8d6dc4984425 "HTTP/1.1 200 OK"                                       

[06/25/24 08:45:42] INFO     HTTP Request: GET                                                      ]8;id=313287;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=742567;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.github.com/repos/apolloconfig/apollo/git/blobs/5abdcf56791                
                             4fd549e12e65e369c8108a9c2791c "HTTP/1.1 200 OK"                                       

185
